In [1]:
import pandas as pd
import numpy as np
from utils import encode_data_get_embeddings, create_emoji_sentiment
import model as models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense
from sklearn.metrics import f1_score, classification_report
import tensorflow as tf

## Nacitanie dat a embedingov

### 1. DATA-EN-TW

In [2]:
# tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data.csv')
tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data_emoji.csv')

### 2. DATA-EN-SE

In [2]:
# tweets = pd.read_csv('D:\Downloads\DP\data\emocontext.csv')
tweets = pd.read_csv('D:\Downloads\DP\data\emocontext_emoji.csv')

# tweets_train = pd.read_csv('D:/Downloads/DP/data/emocontext.csv')
# tweets_dev = pd.read_csv('D:/Downloads/DP/data/emocontext_dev.csv')
# tweets_test = pd.read_csv('D:/Downloads/DP/data/emocontext_test.csv')

### 3. DATA-SK-FB

In [2]:
# tweets = pd.read_csv('D:/Downloads/DP/data/seesame.csv')
tweets = pd.read_csv('D:/Downloads/DP/data/seesame_emoji.csv')

In [3]:
w2v = open('D:\Downloads\DP\word2vec300_en.vec', encoding="utf8")
w2v_dim = w2v.readline()
print(w2v_dim)

e2v = open('D:\Downloads\DP\emoji2vec300.txt', encoding="utf8")
e2v_dim = e2v.readline()
print(e2v_dim)

train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, None)
# train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, tw_cnt, w2v, e2v)

2000000 300

1661 300



## Predikcie - Neuronove siete

### BiLstm - DATA-EN-SE

In [ ]:
classes = ['others', 'sad', 'happy', 'angry']
predict = model.predict(test_x)
predict = np.argmax(predict, axis=1)
predict = [classes[pr] for pr in predict]

In [ ]:
model = Sequential()
model.add(Embedding(embedding_matrix.shape[0],
                 300,
                 weights=[embedding_matrix],
                 input_length=80,
                 trainable=False))

model.add(Bidirectional(LSTM(units=256,
            dropout=0.1,
            recurrent_dropout=0.1
                            )))

model.add(Dense(1, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(train_x, train_y, 
          batch_size=64,
          epochs=10,
          validation_split=0.2)

In [ ]:
print(f1_score(tweets.Label[tw_cnt:], predict, average='micro', labels=['sad', 'happy', 'angry']))
print(classification_report(tweets.Label[tw_cnt:], predict))


### BiLstm - DATA-EN-TW / DATA-SK-FB


In [ ]:
tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data_emoji.csv')

w2v = open('D:\Downloads\DP\word2vec300_en.vec', encoding="utf8")
w2v_dim = w2v.readline()
print(w2v_dim)

e2v = open('D:\Downloads\DP\emoji2vec300.txt', encoding="utf8")
e2v_dim = e2v.readline()
print(e2v_dim)

train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, None)
# train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, e2v)

In [ ]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

## BiLstm model

In [9]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

Epoch 1/10
302/302 [==============================] - 117s 389ms/step - loss: 0.8910 - acc: 0.6539 - val_loss: 0.6864 - val_acc: 0.7261
Epoch 2/10
302/302 [==============================] - 104s 343ms/step - loss: 0.5986 - acc: 0.7704 - val_loss: 0.5821 - val_acc: 0.7793
Epoch 3/10
302/302 [==============================] - 105s 348ms/step - loss: 0.5073 - acc: 0.8099 - val_loss: 0.5337 - val_acc: 0.7984
Epoch 4/10
302/302 [==============================] - 105s 348ms/step - loss: 0.4685 - acc: 0.8220 - val_loss: 0.5300 - val_acc: 0.8040
Epoch 5/10
302/302 [==============================] - 105s 349ms/step - loss: 0.4275 - acc: 0.8405 - val_loss: 0.4784 - val_acc: 0.8257
Epoch 6/10
302/302 [==============================] - 106s 350ms/step - loss: 0.3968 - acc: 0.8557 - val_loss: 0.4467 - val_acc: 0.8361
Epoch 7/10
302/302 [==============================] - 106s 350ms/step - loss: 0.3733 - acc: 0.8645 - val_loss: 0.4531 - val_acc: 0.8373
Epoch 8/10
302/302 [============================

In [10]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: 0.430917888879776, acc: 0.8461538553237915


## Testing model

In [ ]:
model = models.TestModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=32,
    validation_split=0.2)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

### Combi model

In [ ]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=32,
    validation_split=0.2)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

### Conv model

In [ ]:
model = models.ConvModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=5,
    batch_size=32,
    validation_split=0.2)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

### Baseline model

In [ ]:
model = models.BaselineModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=5,
    batch_size=32,
    validation_split=0.2)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

### BaselineDrop model

In [ ]:
model = models.BaselineDropModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=5,
    batch_size=32,
    validation_split=0.2)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

### Lstm model

In [ ]:
model = models.LstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

### Testing

In [66]:
embedding_matrix.shape

(101147, 300)

In [73]:
embedding_matrix.shape

(12228, 300)

In [10]:
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import TweetTokenizer

tw_tok = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

x = np.array((tweets['Text']).apply(lambda x: ' '.join([w for w in tw_tok.tokenize(x)])))

# One-hot encoding labelu
y = np.array(tweets['Label'].apply(lambda x: 2 if x == 'Positive' else (1 if x =='Neutral' else 0)))
#     y = np.array(tweets['Label'].apply(lambda x: 1 if x == 'Positive' else 0))

tweet_count = len(tweets)
limit = int(0.8 * tweet_count)

train_x = x[:limit]
train_y = y[:limit]
test_x = x[limit:]
test_y = y[limit:]

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_x)

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

vocab_size = len(tokenizer.word_index) + 1

In [11]:
vocab_size

59945

In [12]:
embedding_matrix.shape[0]

59991